# Deep Q Network

### Import Dependencies

In [1]:
import os
import numpy as np
import gym

from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

import random

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


### Set parameters

In [2]:
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
batch_size = 32
n_episodes = 1001
output_dir = 'model_output/cartpole'

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gym/utils/passive_env_checker.py:98: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  "We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) "


In [3]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### Define DQN agent

In [4]:
class DQNAgent:
    
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size 
        
        self.memory = deque(maxlen=2000) # use deque to automatically remove earliest elements during insertion
        
        self.gamma = 0.95 # factor future rewards
        
        # regulate explore vs exploit
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        
        self.learning_rate = 0.001
        self.model = self._build_model()
        
    def _build_model(self):
        # use sequential model with three layers
        model = Sequential(
            [
                Dense(24, input_dim = self.state_size, activation = 'relu'),
                Dense(24, activation = 'relu'),
                Dense(self.action_size, activation = 'linear')
            ]
        )
        
        model.compile(
            loss = 'mse',
            optimizer = Adam(learning_rate = self.learning_rate)
        )
        
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def act(self,state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])
    
    def replay(self, batch_size):
        # randomly sample instead of choosing consecutive ones because the correlation between neighbor states
        minibatch = random.sample(self.memory, batch_size)
        
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                # most important part from the dqn paper algorithm
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
                target_f = self.model.predict(state)
                target_f[0][action] = target
                
                self.model.fit(state, target_f, epochs = 1, verbose = 0)
            
            # multiply epsilon by decay rate
            if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay
                
    def load(self, name):
        self.model.load_weights(name)
        
    def save(self, name):
        self.model.save_weights(name)

In [5]:
agent = DQNAgent(state_size, action_size)

2022-06-28 21:25:46.945314: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
done = False

for e in range(n_episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    
    for time in range(5000):
        # env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}".format(e, n_episodes, time, agent.epsilon))
            break
    
    if len(agent.memory) > batch_size:
        agent.replay(batch_size)
        
    if e % 50 == 0:
        agent.save(output_dir + "weights_" + '{:04d}'.format(e) + '.hdf5')

episode: 0/1001, score: 28, e: 1.0
episode: 1/1001, score: 27, e: 1.0
1/1 [==============================] - 0s 19ms/step
episode: 2/1001, score: 13, e: 0.85
1/1 [==============================] - 0s 19ms/step
episode: 3/1001, score: 23, e: 0.73
1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 44ms/step
episode: 4/1001, score: 11, e: 0.62
1/1 [==============================] - 0s 20ms/step
episode: 5/1001, score: 16, e: 0.53
1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 19ms/step
episode: 6/1001, score: 9, e: 0.45
1/1 [==============================] - 0s 22ms/step
episode: 7/1001, score: 9, e: 0.38
1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 18ms/step
episode: 8/1001, score: 8, e: 0.33
1/1 [==============================] - 0s 19ms/step
episode: 9/1001, score: 12, e: 0.28
1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 19ms/step
episode: 10/1001, score: 8, e: 0.24
1/1 [==============================] - 0s 19ms/step
episode: 11/1001, score: 10, e: 0.2
1/1 [==============================] - 0s 27ms/step
episode: 12/1001, score: 9, e: 0.17
1/1 [==============================] - 0s 23ms/step


1/1 [==============================] - 0s 33ms/step
episode: 13/1001, score: 8, e: 0.15
1/1 [==============================] - 0s 20ms/step


KeyboardInterrupt: 